# Data Preprocess and Merge
The goal of this file is to merge the previous years base data to the new years one and bring over statistics, making everything faster while mantaining a coherent and solid procedure over the years to come, by just changing the file names.

## Imports

In [26]:
import os

import numpy as np
import pandas as pd

## Global variables

In [27]:
# Fantacalcio names to Fantahelp names
fh_to_fc= {
    'Id': 'Id',
    'Role': 'R',
    'Role_M': 'RM',
    'Name': 'Nome',
    'Squad': 'Squadra',
    'Price': 'Qt.A',
    'FVM': 'FVM',
    'Pg': 'Pv',
    'Mv': 'Mv',
    'Mf': 'Fm',
}
unneeded_labels_new_df = ['Qt.I','Qt.I M','Qt.A M','Diff.','Diff.M','FVM M']
unneeded_labels_stats = ['Amm','Ass','Au','Esp','Gf','Gs','Rp','Rc','R+','R-','Rm']

## Load previous year and the new data of players and statistics

In [28]:
# load old dataset players (here you have to put the excel of last season's serie a with stats)
print("Old Dataset")
url_old_dataset = '24-25_trial/copyCsvReal.xlsx'
dataframe_old_data = pd.read_excel(url_old_dataset, header=0, index_col=None)
# summarize shape
print("Shape: " + str(dataframe_old_data.shape))
# summarize first few lines
print("Summary dataset")
print(dataframe_old_data)

# load new dataset players
print("New Dataset")
url_players = '24-25/Quotazioni_Fantacalcio_Stagione_2024_25.xlsx'
dataframe_players = pd.read_excel(url_players, header=1, index_col=None)
# drop unneeded columns
dataframe_players = dataframe_players.drop(unneeded_labels_new_df, axis=1)
# summarize shape
print("Shape: " + str(dataframe_players.shape))
# summarize first few lines
print("Summary dataset")
print(dataframe_players)

# load statistics
print("Loading Statistics from previous years")
folder_path = 'stats'
# List all files in the folder
file_list = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
stats_dataframes = {}

# Iterate over the file list and load each file into a DataFrame
for file in file_list:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, header=1, index_col=None)
    # drop unneeded columns
    df = df.drop(unneeded_labels_stats, axis=1)
    
    # Store the DataFrame in the dictionary with the file name (without extension) as the key
    file_name = os.path.splitext(file)[0]
    stats_dataframes[file_name] = df

# If you want to see the DataFrames, you can print them out
for name, df in stats_dataframes.items():
    print(f"DataFrame from {name}:\n", df.head(), "\n")

Old Dataset
Shape: (539, 25)
Summary dataset
       Id Role         Name     Squad  Price  MyRating         Mate  \
0    4431    P  Carnesecchi  Atalanta     10       NaN        Musso   
1    2792    P        Musso  Atalanta      5       NaN  Carnesecchi   
2    2297    P     Rossi F.  Atalanta      1       NaN          NaN   
3     554    D   Zappacosta  Atalanta     15       NaN         Holm   
4    5067    D       Bakker  Atalanta     11       NaN      Ruggeri   
..    ...  ...          ...       ...    ...       ...          ...   
534  5529    A        Henry    Verona     11       NaN          NaN   
535  5471    A       Djuric    Verona     10       NaN    Bonazzoli   
536   505    A    Bonazzoli    Verona      8       NaN       Djuric   
537  5395    A        Braaf    Verona      1       NaN          NaN   
538  5439    A       Kallon    Verona      1       NaN          NaN   

     Regularness  FVM  Age  ...  Mf21_22  Pg20_21  Mv20_21  Mf20_21  Pg19_20  \
0              3   10 

## Load and Merge
all data in a new dataframe

**Create new dataframe**


In [29]:
column_names = ['Id','Role', 'Role_M','Name','Squad','Price','Age','MyRating','Mate','Regularness','FVM']
stats_columns = []
for name, df in stats_dataframes.items():
    season = name[-5:]
    stats_columns.append(f"Mf{season}")
    stats_columns.append(f"Mv{season}")
    stats_columns.append(f"Pg{season}")
stats_columns = reversed(stats_columns)
column_names.extend(stats_columns)
# Create a DataFrame filled with column values
new_df = pd.DataFrame(columns=column_names)
print(df)

       Id  R         Nome    Squadra  Pv    Mv    Fm
0    2428  P       Sommer      Inter  34  6.12  5.63
1     453  P     Szczesny   Juventus  35  6.21  5.43
2    5841  P       Svilar       Roma  15  6.47  5.37
3    5876  P  Di Gregorio      Monza  33  6.35  5.47
4     133  P    Skorupski    Bologna  32  6.17  5.33
..    ... ..          ...        ...  ..   ...   ...
659  6490  A         Cruz     Verona   3  5.67  5.67
660  6519  A      Camarda      Milan   0  0.00  0.00
661  6530  A    Ghedjemis  Frosinone   5  6.20  6.10
662  6565  A       Ankeye      Genoa   2  5.75  5.75
663  6583  A        Kabic     Torino   0  0.00  0.00

[664 rows x 7 columns]


**Load rows**

In [30]:
for index, row in dataframe_players.iterrows():
    id = row[fh_to_fc['Id']]
    old_row = dataframe_old_data[dataframe_old_data['Id'] == id]
    role = row[fh_to_fc['Role']]
    role_m = row[fh_to_fc['Role_M']]
    name = row[fh_to_fc['Name']]
    squad = row[fh_to_fc['Squad']]
    price = row[fh_to_fc['Price']]
    age = np.nan if old_row.empty else int(old_row['Age']) + 1
    fvm = row[fh_to_fc['FVM']]
    results = {}

    for key, df in stats_dataframes.items():
        season = key[-5:]
        if id in df['Id'].values:
            # Filter the DataFrame for the specific Id
            row = df[df['Id'] == id]
            
            # Extract values from the relevant columns
            results[f"Pg{season}"] = row[fh_to_fc["Pg"]].values[0]
            results[f"Mv{season}"] = row[fh_to_fc["Mv"]].values[0]
            results[f"Mf{season}"] = row[fh_to_fc["Mf"]].values[0]
        else:
            results[f"Pg{season}"] = np.nan
            results[f"Mv{season}"] = np.nan
            results[f"Mf{season}"] = np.nan
            
    new_row = {
        'Id': id,
        'Role': role,
        'Role_M': role_m,
        'Name': name,
        'Squad': squad,
        'Price': price,
        'Age': age,
        'MyRating': np.nan,
        'Mate': np.nan,
        'Regularness': np.nan,
        'FVM': fvm
    }
    new_row = new_row | results
    new_row = pd.DataFrame([new_row])
    new_df = pd.concat([new_df, new_row], ignore_index=True)
print(new_df)

C:\Users\d.cirino\AppData\Local\Temp\ipykernel_4708\2901837039.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  age = np.nan if old_row.empty else int(old_row['Age']) + 1
C:\Users\d.cirino\AppData\Local\Temp\ipykernel_4708\2901837039.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, new_row], ignore_index=True)
C:\Users\d.cirino\AppData\Local\Temp\ipykernel_4708\2901837039.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  age = np.nan if old_row.empty else int(old_row['Age']) + 1
C:\Users\d.cirino\AppData\Local\Temp\ip

       Id Role Role_M         Name     Squad Price  Age MyRating Mate  \
0    2428    P    Por       Sommer     Inter    19   35      NaN  NaN   
1    5876    P    Por  Di Gregorio  Juventus    18   27      NaN  NaN   
2     572    P    Por        Meret    Napoli    15   27      NaN  NaN   
3    4312    P    Por      Maignan     Milan    14   29      NaN  NaN   
4    4431    P    Por  Carnesecchi  Atalanta    13   24      NaN  NaN   
..    ...  ...    ...          ...       ...   ...  ...      ...  ...   
524  6598    A     Pc     Mutandwa  Cagliari     1  NaN      NaN  NaN   
525  6603    A      A   Joao Costa      Roma     1  NaN      NaN  NaN   
526  6648    A    W;A        Jasim      Como     1  NaN      NaN  NaN   
527  6670    A     Pc  Charpentier     Parma     1  NaN      NaN  NaN   
528  6822    A     Pc      Ekhator     Genoa     1  NaN      NaN  NaN   

    Regularness  ... Mf22_23 Pg21_22  Mv21_22  Mf21_22 Pg20_21  Mv20_21  \
0           NaN  ...     NaN     NaN      NaN   

C:\Users\d.cirino\AppData\Local\Temp\ipykernel_4708\2901837039.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  age = np.nan if old_row.empty else int(old_row['Age']) + 1
C:\Users\d.cirino\AppData\Local\Temp\ipykernel_4708\2901837039.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  age = np.nan if old_row.empty else int(old_row['Age']) + 1
C:\Users\d.cirino\AppData\Local\Temp\ipykernel_4708\2901837039.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  age = np.nan if old_row.empty else int(old_row['Age']) + 1
C:\Users\d.cirino\AppData\Local\Temp\ipykernel_4708\2901837039.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.il

## Save the output of the preprocessing

In [31]:
new_df.to_excel(f"24-25/data_preprocess_merge.xlsx", index=False)

Now remember to update the missing ages and regularness before passing to the ratings generation.